In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *

In [ ]:
casenames = dict(
    c1  = 'R11_MSM97-CT05_f0',
    c2  = 'R11_MSM97-CT05_Stokes6_f0',
    c3  = 'R11_MSM97-CT05_Stokes_f0',
    c4  = 'R11_MSM97-CT05_Stokes5_f0',
    c5  = 'R11_MSM97-CT_f0',
    c6  = 'R11_MSM97-CT_Stokes6_f0',
    c7  = 'R11_MSM97-CT_Stokes_f0',
    c8  = 'R11_MSM97-CT_Stokes5_f0',
    c9  = 'R11_MSM97-CT5h_f0',
    c10 = 'R11_MSM97-CT5h_Stokes6_f0',
    c11 = 'R11_MSM97-CT5h_Stokes_f0',
    c12 = 'R11_MSM97-CT5h_Stokes5_f0',
)
snapshot = '2000-01-02T00:00:00'
abc = 'abcdefghijkl'
figpath = 'R11_MSM97_CT_Stokes_f0'
os.makedirs(figpath, exist_ok=True)

In [ ]:
var = 'w'
da = {}
for cn in casenames.keys():
    print(cn)
    datapath = os.path.join(os.path.pardir, 'oceananigans', casenames[cn], 'fields.jld2')
    da[cn] = OceananigansDataVolume(filepath=datapath, fieldname=var).dataset.data_vars[var]

In [ ]:
depth = 1.5
levels = [np.linspace(-0.01, 0.01, 41), np.linspace(-0.02, 0.02, 41), np.linspace(-0.04, 0.04, 41)]
nrow = 3
ncol = 4
labels = ['5 W m$^{-2}$','50 W m$^{-2}$','500 W m$^{-2}$']
labels2 = ['$\\times 2$', None, '$\\times 0.5$', None]
titles = ['$a$ = {:4.2f} m'.format(amp) for amp in [0,0.45,0.8,1.42]]
fig, axarr = plt.subplots(nrow, ncol, sharex='col', sharey='row')
fig.set_size_inches(8,8/ncol*nrow+1)
im = [np.nan,np.nan,np.nan]
for i, cn in enumerate(casenames.keys()):
    m = i//ncol
    n = i%ncol
    ax = axarr[m,n]
    im[m] = xr.plot.contourf(da[cn].sel(zi=-depth, time=snapshot, method='nearest'), ax=ax, y='y', levels=levels[m], extend='both', add_colorbar=False)
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_aspect('equal')
    if n == 0:
        ax.text(0.05, 0.05, '({:s}) {:s}'.format(abc[i], labels[m]), transform=ax.transAxes, va='bottom', ha='left', bbox=dict(facecolor='w', alpha=0.75, edgecolor='none'))
    else:
        ax.text(0.05, 0.05, '({:s})'.format(abc[i]), transform=ax.transAxes, va='bottom', ha='left', bbox=dict(facecolor='w', alpha=0.75, edgecolor='none'))
    ax = axarr[m,0]
    ax.set_ylabel('$y$ [m]')
    if labels2[m] is not None:
        ax.text(0.05, 0.95, '{:s}'.format(labels2[m]), transform=ax.transAxes, va='top', ha='left', bbox=dict(facecolor='w', alpha=0.75, edgecolor='none'))
for j in np.arange(ncol):
    ax = axarr[0,j]
    ax.set_title(titles[j], fontsize=10)
    ax = axarr[-1,j]
    ax.set_xlabel('$x$ [m]')

plt.subplots_adjust(top=0.95, bottom=0.18, left=0.08, right=0.97, hspace=0.08, wspace=0.06)

cax = plt.axes([0.25, 0.08, 0.5, 0.015])
cb = plt.colorbar(im[1], cax=cax, orientation='horizontal')
clabels = np.linspace(levels[1][0], levels[1][-1], 5)
cb.set_ticks(clabels) 
cb.set_label('$w$ [m s$^{-1}$]')

figname = os.path.join(figpath, 'snapshots-stratification-{:s}'.format(str(depth).replace('.', 'p')))
fig.savefig(figname, dpi = 300, facecolor='w')